In [1]:
import numpy as np
import pandas as pd
import os
import torch

os.chdir("../")

from src.conf import LAYERS_DIMS, MODEL_FEATURES
from src.utils import get_device
from src.data import DataLoaders
from src.models import FraudAutoEncoder
from src.models import Model_Inference

## Data Gen

In [2]:
n_, k_ = 1_000, 300

In [3]:
pd.DataFrame(
    data=np.random.uniform(** {"low": -1, "high": 1, "size": [n_, k_]}),
    columns=[f"X_{str(i).zfill(3)}" for i in range(1, 1+k_)]
).to_parquet("./data/simulated_raw_data_new_arrival.gzip", compression="gzip")

In [4]:
X = pd.read_parquet("/home/onyxia/work/fraud_detection/data/simulated_raw_data_new_arrival.gzip")

## Model Loading

In [5]:
Model_Inference("simulated_data")._error_eval(X.values[98])

0.31623148918151855